# Exploring the Relationship Between the Meta Scores of Films and the Reviews of the General Audiance


## Introduction:
IMDB is an online database for films, television series, podcasts, and other media. This project will be working with movie data from IMDB. The data set of interest contains content data - including cast, production crew, plot summary, and scores. IMDB presents two kinds of scores; IMDB scores which are user-generated averages of reviews submitted to IMDB by the general audience, and Meta Scores which are critic-generated, representing the views of professionals who analyze films.

The goal of this project is to answer the following question:
"How accurate are the Meta Scores of films in predicting the reception of the general audience?"


The data set is from Kaggle, and includes data from the top 1000 movies based on the IMDB score. The data set includes the following columns:
* Series_Title: the name of the film
* Released_year: year it was released
* Certificate: certificate earned by the movie
* Runtime - total runtime
* Genre - list of genres the film falls into
* IMDB_Rating - the IMDB rating given by IMDB user reviews
* Overview - plot summary
* Meta_score - meta score of the movie determined by movie critics
* Director - name of the director of the movie
* Star1, Star2, Star3, Star4 - names of the stars of the movie in order of significance
* No_of_votes - number of reviews on IMDB
* Gross - how much money the movie earned

Dataset Origin: https://www.kaggle.com/datasets/harshitshankhdhar/imdb-dataset-of-top-1000-movies-and-tv-shows

## Methods:
The data will first be prepared by filtering for films released after 1970 and removing films without values in the Meta_score column. The relationship will be determined with a training data set containing 75% of the films a k-nn regression model using Meta_scores as the predictive variable and IMDB_Rating as the response variable. The k value will be determined through a tuning process to produce the minimal error between the training data set and the testing data set containing the films not found in the training data set. The error in the regression produced will give an indication on how strong the relationship is between the two variables. 

To visualize the result the scatter plot will be overlaid with a line produced by the regression to show whether the data is positively or negatively correlated and how much spread there is from the line and the data points.

Tidyverse, dplyr, and tidymodels libraries were imported for data manipulation and visualization.

In [ ]:
### Initialize libraries. 
library(tidyverse)
library(dplyr)
library(tidymodels)
options(repr.matrix.max.rows = 8)

The data was downloaded from: https://www.kaggle.com/datasets/harshitshankhdhar/imdb-dataset-of-top-1000-movies-and-tv-shows
then added to this project's repository. The following cell reads the data from the repository as a CSV file and filters for films released after 1970, as these films will represent modern conditions better.

In [ ]:
### Reading from Github
url <- "https://github.com/anh-dong/dsci-100-2023w1-group-33/blob/main/data/imdb_top_1000.csv?raw=true"
movies <- read_csv(url) |>
    filter(Released_Year > 1970) |>
    select(IMDB_Rating, Meta_score) |>
    na.omit()

The raw data is then split into a training and testing set of 75% and 25% of the raw data respectively.

In [ ]:
### Training data
set.seed(1234)

movies_split <- initial_split(movies, prop = 0.75, strata = Meta_score)
movies_training <- training(movies_split)
movies_testing <- testing(movies_split)

A plot of IMDB_Rating vs Meta_score was created using ggplot and geom_point. Since there was significant overlap between many data points, an alpha of ___ was used to visualize all the data.

In [ ]:
options(repr.plot.height = 8, repr.plot.width = 7)
# your code here
movies_eda <- movies_training |>
    ggplot(aes(x = Meta_score, y = IMDB_Rating)) +
        geom_point(alpha = 0.2) +
        xlab("Meta Sc
ore") +
        ylab("IMDB Rating") +
        ggtitle("IMDB Rating vs Meta Score")
movies_eda

To best determine the relationship between the IMDB Rating and the Meta Score a linear regression was performed using Meta_score as the predictor and IMDB Rating as the response variable. The training data was fit to this workflow to predict parameters for the y-intercept and slope of the regression line.

In [ ]:
lm_spec <- linear_reg() |>
  set_engine("lm") |>
  set_mode("regression")

# TODO: Swap variables
lm_recipe <- recipe(IMDB_Rating ~ Meta_score, data = movies_training)

lm_fit <- workflow() |>
  add_recipe(lm_recipe) |>
  add_model(lm_spec) |>
  fit(data = movies_training)
lm_fit

The linear regression line was then plotted against the training data to visualize how well it fits the data.

In [ ]:
movies_preds <- lm_fit |>
   predict(movies_training) |>
   bind_cols(movies_training)

lm_predictions <- movies_preds |>
    ggplot(aes(x = Meta_score, y = IMDB_Rating)) +
        geom_point(alpha = 0.4) +
        geom_line(
            mapping = aes(x = Meta_score, y = .pred), 
            color = "blue") +
        ggtitle("Linear Regression of Meta Score vs. IMDB Rating Score")+
        xlab("Meta Score")+
        ylab("IMDB Rating Score")+
        theme(text = element_text(size = 20))
lm_predictions

In [ ]:
# TODO: We should calculate this, it will help us determine how good our prediction is.
lm_test_results <- lm_fit |>
         predict(movies_testing) |>
         bind_cols(movies_testing) |>
         metrics(truth = Meta_score, estimate = .pred)

lm_rmspe <- lm_test_results |>
          filter(.metric == "rmse") |>
          select(.estimate) |>
          pull()


lm_rmspe

In [ ]:
lm_training_results <- lm_fit |>
         predict(movies_training) |>
         bind_cols(movies_training) |>
         metrics(truth = Meta_score, estimate = .pred)

lm_rmse <- lm_training_results |>
          filter(.metric == "rmse") |>
          select(.estimate) |>
          pull()

lm_rmse

The testing data set was then compared to their predictions based on our model and plotted against the line of best fit calculated above.

In [ ]:
test_preds <-  lm_fit |>
   predict(movies_testing) |>
   bind_cols(movies_testing)

lm_predictions_test <- test_preds |>
     ggplot(aes(x = Meta_score, y =IMDB_Rating )) +
         geom_point(alpha = 0.4) +
         geom_line(
             mapping = aes(x = Meta_score, y = .pred), 
             color = "blue") +
xlab("Meta Score")+
ylab("IMDB Rating Score")+
         theme(text = element_text(size = 20))
lm_predictions_test
#TODO: in general we should try and get rid of the warning messages. I think this should be done when we remove NA values from our training/testing set.

## Expected Outcomes and Significance:
It is expected that the Meta Scores and audience reception will have a weak positive relationship. Critics are trying to give audiences an accurate expectation on the film based on their ratings so they should correlate positively with each other. However, critics and general audiences often look for different things in their ratings, producing more variability between the two ratings, potentially causing a weaker relationship. This relationship will allow audiences to understand how a meta score should factor into their decision to see a movie before any general audience ratings are available.